<a href="https://colab.research.google.com/github/reponseashimwe/dqn-atari/blob/main/FM3_Atari_Environments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ⚙️ Installation & Imports
Run this cell first. It installs all dependencies and handles the necessary environment registration.

In [6]:
# 1. INSTALLATION, REGISTRATION, AND IMPORTS

# Install core DRL libraries
!pip install stable-baselines3[extra] gymnasium[atari] ale-py -q
# Install OpenCV and AutoROM (required dependencies for Atari wrappers)
!pip install opencv-python autorom[accept-rom-license] -q

import gymnasium as gym
import ale_py
import os
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.atari_wrappers import AtariWrapper
from collections import defaultdict
import gc

# FIX: Explicitly register the ALE environments to avoid NamespaceNotFound
gym.register_envs(ale_py)

print("Setup complete. Running core imports.")


Setup complete. Running core imports.


# 2. 📁 Drive Setup and Experiment Definitions
This cells handles mounting the drive and defines the five hyperparameter sets, including the one that achieved your best result (Set 1).

In [2]:
# Drive Mount
from google.colab import drive

# --- Drive Configuration ---
# Mount Drive first
drive.mount('/content/drive')

DRIVE_PATH = '/content/drive/MyDrive'
ASSIGNMENT_FOLDER = 'DQN_Breakout_Final_Submission'
BASE_DEST_PATH = f'{DRIVE_PATH}/{ASSIGNMENT_FOLDER}/Experiments'
os.makedirs(BASE_DEST_PATH, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# --- Global DRL Configuration ---
ENV_ID = "BreakoutNoFrameskip-v4"
TOTAL_TIMESTEPS = 1_000_000
N_ENVS = 4
TEMP_MODEL_DIR = "./temp_models/"
os.makedirs(TEMP_MODEL_DIR, exist_ok=True)

# Dictionary to store results for final comparison
results_comparison = defaultdict(lambda: {'avg_reward': 0, 'params': {}})

# --- Hyperparameter Definitions (5 Sets) ---
HYPER_SETS = {
    # Set 1: Baseline
    'Set_1_Best_Policy': {
        'lr': 1e-4, 'gamma': 0.99, 'batch_size': 32,
        'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1
    },
    # Set 2: High LR
    'Set_2_High_LR': {
        'lr': 5e-4, 'gamma': 0.99, 'batch_size': 32,
        'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1
    },
    # Set 3: Low Gamma
    'Set_3_Low_Gamma': {
        'lr': 1e-4, 'gamma': 0.90, 'batch_size': 32,
        'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1
    },
    # Set 4: Extended Exploration
    'Set_4_Extended_Eps': {
        'lr': 1e-4, 'gamma': 0.99, 'batch_size': 32,
        'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.5
    },
    # Set 5: Larger Batch Size (New experiment for robust tuning)
    'Set_5_Large_Batch': {
        'lr': 1e-4, 'gamma': 0.99, 'batch_size': 128,
        'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1
    }
}
print("Experiment definitions complete. Starting training runs.")

Experiment definitions complete. Starting training runs.


#3. 🧪 Training, Evaluation, and Drive Saving Loop (Core Logic)
This cell defines the functions and runs all five experiments, saving each to a distinct Drive folder.

In [12]:
def setup_environment(env_id, n_envs):
    env = make_atari_env(env_id, n_envs=n_envs, seed=np.random.randint(1000))
    env = VecFrameStack(env, n_stack=4)
    return env

def evaluate_agent(model, env_id, n_episodes=10):
    """Runs evaluation episodes, PRINTS individual results, and returns average."""
    eval_env = setup_environment(env_id, n_envs=1)
    episode_rewards = []

    print(f"\nStarting evaluation ({n_episodes} episodes)...")

    for i in range(n_episodes):
        obs = eval_env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done_array, info = eval_env.step(action)
            total_reward += reward[0]
            done = done_array[0]
        episode_rewards.append(total_reward)

        # --- PRINT INDIVIDUAL EPISODE ---
        print(f"Episode {i+1} finished with reward: {total_reward:.2f}")

    eval_env.close()
    mean_reward = np.mean(episode_rewards)

    # --- PRINT AVERAGE ---
    print(f"--> Average Reward over {n_episodes} episodes: {mean_reward:.2f}")

    return mean_reward

def run_single_experiment(set_name):
    """Runs ONE experiment based on the set name key."""
    hyperparams = HYPER_SETS[set_name]

    print(f"\n" + "="*60)
    print(f"🚀 STARTING EXPERIMENT: {set_name}")
    print(f"Parameters: {hyperparams}")
    print(f"="*60)

    # Paths
    LOG_DIR = os.path.join(TEMP_MODEL_DIR, f"{set_name}_logs")
    TEMP_MODEL_PATH = os.path.join(TEMP_MODEL_DIR, f"{set_name}.zip")
    DRIVE_DEST_FOLDER = os.path.join(BASE_DEST_PATH, set_name)
    os.makedirs(DRIVE_DEST_FOLDER, exist_ok=True)

    # Init Environment & Model
    env = setup_environment(ENV_ID, N_ENVS)

    # Callback for safety
    checkpoint_callback = CheckpointCallback(
        save_freq=200000 // N_ENVS, save_path=LOG_DIR, name_prefix="dqn_chkpt"
    )

    model = DQN(
        "CnnPolicy", env,
        learning_rate=hyperparams['lr'], gamma=hyperparams['gamma'],
        batch_size=hyperparams['batch_size'],
        exploration_initial_eps=hyperparams['epsilon_start'],
        exploration_final_eps=hyperparams['epsilon_end'],
        exploration_fraction=hyperparams['epsilon_decay'],
        buffer_size=50000,
        train_freq=4,
        verbose=0, # <--- CHANGED: 0 hides tables, but keeps progress bar below
        tensorboard_log=LOG_DIR
    )

    # Train with Progress Bar
    print("⏳ Training started... (Wait for progress bar)")
    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=checkpoint_callback, progress_bar=True)
    model.save(TEMP_MODEL_PATH)

    # Evaluate
    print(f"\n📊 Evaluating {set_name}...")
    avg_reward = evaluate_agent(model, ENV_ID, n_episodes=10)
    print(f"✅ Evaluation Avg Reward: {avg_reward:.2f}")

    # Save to Drive
    print(f"💾 Copying to Drive Folder: {DRIVE_DEST_FOLDER}...")
    !cp -r "$LOG_DIR" "$DRIVE_DEST_FOLDER/"
    !cp "$TEMP_MODEL_PATH" "$DRIVE_DEST_FOLDER/"
    print("Done.")

    # Cleanup Memory
    env.close()
    del model
    del env
    gc.collect()

#▶️ Execution Cells (Run One by One)

## Run Set 1 (Best Policy)

In [5]:
run_single_experiment('Set_1_Best_Policy')


🚀 STARTING EXPERIMENT: Set_1_Best_Policy
Parameters: {'lr': 0.0001, 'gamma': 0.99, 'batch_size': 32, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1}
⏳ Training started... (Wait for progress bar)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):


📊 Evaluating Set_1_Best_Policy...
✅ Evaluation Avg Reward: 5.50
💾 Copying to Drive Folder: /content/drive/MyDrive/DQN_Breakout_Final_Submission/Experiments/Set_1_Best_Policy...
Done.


NameError: name 'gc' is not defined

### Run Set 2 (High LR)

In [7]:
run_single_experiment('Set_2_High_LR')


🚀 STARTING EXPERIMENT: Set_2_High_LR
Parameters: {'lr': 0.0005, 'gamma': 0.99, 'batch_size': 32, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1}
⏳ Training started... (Wait for progress bar)


Output()


📊 Evaluating Set_2_High_LR...
✅ Evaluation Avg Reward: 4.90
💾 Copying to Drive Folder: /content/drive/MyDrive/DQN_Breakout_Final_Submission/Experiments/Set_2_High_LR...
Done.


### Run Set 3 (Low Gamma)

In [8]:
run_single_experiment('Set_3_Low_Gamma')


🚀 STARTING EXPERIMENT: Set_3_Low_Gamma
Parameters: {'lr': 0.0001, 'gamma': 0.9, 'batch_size': 32, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1}
⏳ Training started... (Wait for progress bar)


Output()


📊 Evaluating Set_3_Low_Gamma...
✅ Evaluation Avg Reward: 0.80
💾 Copying to Drive Folder: /content/drive/MyDrive/DQN_Breakout_Final_Submission/Experiments/Set_3_Low_Gamma...
Done.


### Run Set 4 (Extended Exploration)

In [9]:
run_single_experiment('Set_4_Extended_Eps')


🚀 STARTING EXPERIMENT: Set_4_Extended_Eps
Parameters: {'lr': 0.0001, 'gamma': 0.99, 'batch_size': 32, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.5}
⏳ Training started... (Wait for progress bar)


Output()


📊 Evaluating Set_4_Extended_Eps...
✅ Evaluation Avg Reward: 3.70
💾 Copying to Drive Folder: /content/drive/MyDrive/DQN_Breakout_Final_Submission/Experiments/Set_4_Extended_Eps...
Done.


### Run Set 5 (Large Batch)

In [10]:
run_single_experiment('Set_5_Large_Batch')


🚀 STARTING EXPERIMENT: Set_5_Large_Batch
Parameters: {'lr': 0.0001, 'gamma': 0.99, 'batch_size': 128, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay': 0.1}
⏳ Training started... (Wait for progress bar)


Output()


📊 Evaluating Set_5_Large_Batch...
✅ Evaluation Avg Reward: 6.80
💾 Copying to Drive Folder: /content/drive/MyDrive/DQN_Breakout_Final_Submission/Experiments/Set_5_Large_Batch...
Done.


#4. 🏆 Final Selection and Play Script Setup
This cell determines the best model path and sets up the final play.py environment to use it.

In [13]:
# 4. FINAL SELECTION AND PLAY SCRIPT SETUP

FINAL_MODEL_PATH_COLAB = "dqn_final_best_model.zip"

# 1. Determine the Best Model
best_set_name = None
max_reward = -np.inf

for name, result in results_comparison.items():
    print(f"- {name}: Avg Reward = {result['avg_reward']:.2f}")
    if result['avg_reward'] > max_reward:
        max_reward = result['avg_reward']
        best_set_name = name

best_model_path_drive = results_comparison[best_set_name]['model_path']

# 2. Copy the best model back to Colab root for the play script
# (Since the play script assumes the model is in the root directory)
!cp "$best_model_path_drive" "$FINAL_MODEL_PATH_COLAB"

print("\n" + "="*70)
print(f"🎉 Final Best Policy: {best_set_name} with Average Reward: {max_reward:.2f}")
print(f"The best model has been copied to: {FINAL_MODEL_PATH_COLAB}")
print("="*70)


# --- 5. PLAY SCRIPT (Task 2) ---

def make_env_render():
    # Use a standard Gym environment for rendering the GUI
    env = gym.make(ENV_ID, render_mode="human")
    env = AtariWrapper(env)
    return env

def play_agent(model_path):
    print("\nStarting agent evaluation (GreedyQPolicy) for FINAL BEST MODEL...")

    try:
        # Load the BEST model
        model = DQN.load(model_path)
    except FileNotFoundError:
        print(f"Error: Final model not found at {model_path}. Cannot play.")
        return

    # Use a single wrapped VecEnv for consistency, but with the human renderer
    eval_env = DummyVecEnv([make_env_render])
    eval_env = VecFrameStack(eval_env, n_stack=4)

    for i in range(5): # Run 5 episodes for visualization
        obs = eval_env.reset()
        done = False
        total_reward = 0

        while not done:
            # Predict the action with deterministic=True (GreedyQPolicy)
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done_array, info = eval_env.step(action)

            # Rendering is handled by the "human" render_mode
            eval_env.render()

            done = done_array[0]
            total_reward += reward[0]

            if done:
                break

        print(f"Visual Episode {i+1} finished with reward: {total_reward:.2f}")

    eval_env.close()
    print("Evaluation complete.")


if __name__ == "__main__":
    play_agent(FINAL_MODEL_PATH_COLAB)

- None: Avg Reward = 0.00


KeyError: 'model_path'